In [1]:
import tkinter as tk
from tkinter import messagebox, ttk
import csv
import mysql.connector as sql
from PIL import Image, ImageTk

Login Window

In [2]:
class LoginWindow:
    def __init__(self, root, on_successful_login):
        self.root = root
        self.root.title("Login to Swiggy Restaurant Data")
        self.on_successful_login = on_successful_login

        # Load and set background image
        try:
            self.bg_image = ImageTk.PhotoImage(Image.open("bg_LOGIN.jpg").resize((1280, 720), Image.Resampling.LANCZOS))
            self.bg_label = tk.Label(self.root, image=self.bg_image)
            self.bg_label.place(x=0, y=0, relwidth=1, relheight=1)
        except Exception as e:
            messagebox.showerror("Error", f"Failed to load background image: {e}")

        # Create the login frame
        self.login_frame = tk.Frame(self.root, bg='white')
        self.login_frame.pack(padx=10, pady=175)

        # Username field
        tk.Label(self.login_frame, text="Username:", bg='white').grid(row=0, column=0, padx=5, pady=5)
        self.username_entry = tk.Entry(self.login_frame)
        self.username_entry.grid(row=0, column=1, padx=5, pady=5)

        # Password field
        tk.Label(self.login_frame, text="Password:", bg='white').grid(row=1, column=0, padx=5, pady=5)
        self.password_entry = tk.Entry(self.login_frame, show="*")
        self.password_entry.grid(row=1, column=1, padx=5, pady=5)

        # Login button
        tk.Button(self.login_frame, text="Login", command=self.attempt_login).grid(row=2, column=0, columnspan=2, pady=10)

    def attempt_login(self):
        username = self.username_entry.get()
        password = self.password_entry.get()

        # Read credentials from CSV file
        credentials = {}
        try:
            with open("credentials.csv", "r") as file:
                csv_reader = csv.DictReader(file)
                for row in csv_reader:
                    credentials[row["user"]] = row["password"]
        except FileNotFoundError:
            messagebox.showerror("Error", "credentials.csv file not found.")
            self.root.quit()
            return
        except Exception as e:
            messagebox.showerror("Error", f"Failed to read credentials: {e}")
            self.root.quit()
            return

        # Check if the username and password match
        if username in credentials and credentials[username] == password:
            self.login_frame.destroy()
            self.on_successful_login(username, password)
        else:
            messagebox.showerror("Login Failed", "Invalid username or password.")

Printing All Tables

In [3]:
class SwiggyApp:
    def __init__(self, root, username, password, show_login):
        self.root = root
        self.root.title("SWIGGY DATA ANALYSIS")
        self.show_login = show_login

        self.bg_image = ImageTk.PhotoImage(Image.open("bg.JPG").resize((1280, 720), Image.Resampling.LANCZOS))
        self.bg_label = tk.Label(self.root, image=self.bg_image)
        self.bg_label.place(x=0, y=0, relwidth=1, relheight=1)

        try:
            self.mydb = sql.connect(
                # host="192.168.194.204",
                host="localhost",
                # user="sagnik",
                user="root",
                #passwd="remote",
                password="root",
                # database="swiggy_python",
                database="new_data",
                port="3306"
            )
            self.mycursor = self.mydb.cursor()
        except sql.Error as e:
            messagebox.showerror("Database Error", f"Failed to connect to database: {e}")
            self.show_login()
            return

        self.main_frame = tk.Frame(self.root, bg='white')
        self.main_frame.pack(padx=10, pady=150)

        tables = ['CITY_DIM', 'TABLE_BOOKING', 'CUISINES', 'DELIVERY', 'FACT_SWIGGY', 'LOCALITY_DIM', 'RATINGS', 'RESTAURANT']

        for table in tables:
            tk.Button(self.main_frame, text=table, command=lambda t=table: self.display_table(t)).pack(fill='x')

        tk.Button(self.main_frame, text="Logout", command=self.logout).pack(fill='x', pady=5)

    def logout(self):
        self.close_connection()
        self.main_frame.destroy()

    def close_connection(self):
        if hasattr(self, 'mydb') and self.mydb.is_connected():
            self.mydb.close()
            self.mycursor.close()

Output Window

In [4]:
class ResultsWindow(tk.Toplevel):
    def __init__(self, main_app, table_name):
        tk.Toplevel.__init__(self)
        self.main_app = main_app
        self.title(f"{table_name} Table")

        self.bg_image = ImageTk.PhotoImage(Image.open("bg.JPG").resize((1280, 720), Image.Resampling.LANCZOS))
        self.bg_label = tk.Label(self, image=self.bg_image)
        self.bg_label.place(x=0, y=0, relwidth=1, relheight=1)

        try:
            self.main_app.mycursor.execute(f"SELECT * FROM {table_name}")
            result = self.main_app.mycursor.fetchall()
            columns = [desc[0] for desc in self.main_app.mycursor.description]

            tree_frame = tk.Frame(self, bg='white')
            tree_frame.pack(expand=True, fill='both')

            tree = ttk.Treeview(tree_frame, columns=columns, show='headings')
            for col in columns:
                tree.heading(col, text=col)
                tree.column(col, width=100)

            for row in result:
                tree.insert('', 'end', values=row)

            scrollbar = tk.Scrollbar(tree_frame, orient="vertical", command=tree.yview)
            tree.configure(yscrollcommand=scrollbar.set)
            tree.pack(side='left', expand=True, fill='both')
            scrollbar.pack(side='right', fill='y')

        except sql.Error as e:
            messagebox.showerror("Query Error", f"Failed to execute query: {e}")
            self.destroy()
            self.main_app.show_main_window()
            return

        back_button = tk.Button(self, text="Back", command=self.go_back)
        back_button.pack(pady=10)

        self.protocol("WM_DELETE_WINDOW", self.go_back)

    def go_back(self):
        self.destroy()
        self.main_app.show_main_window()

In [5]:
def main():
    root = tk.Tk()
    root.geometry("1280x720")

    def show_login():
        for widget in root.winfo_children():
            widget.destroy()
        LoginWindow(root, lambda username, password: SwiggyApp(root, username, password, show_login))

    show_login()

    root.mainloop()

In [6]:
if __name__ == "__main__":
    main()